In [5]:
import cv2
import os
import json
import re
import glob

def sortCriterion(x):
    return int(re.findall(r'\d+', x)[0])

class DatasetLoader:
    """
    DatasetLoader class is used to crop images using annotations in json format.

    Attributes:
        folder_path (str): Path to the folder containing images.
        annotsPath (str): Path to the folder containing annotations in json format.
        output_folder_path (str): Path to the folder where the cropped images will be saved.
        image_files (list): List of paths to the images in `folder_path`.
    """
    
    def __init__(self, folder_path,annotsPath,output_folder_path):
        """
        The constructor for the DatasetLoader class.

        Parameters:
            folder_path (str): Path to the folder containing images.
            annotsPath (str): Path to the folder containing annotations in json format.
            output_folder_path (str): Path to the folder where the cropped images will be saved.
        """
        self.folder_path = folder_path
        self.image_files = sorted(glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png')),
                                  key=sortCriterion)
        self.annotsPath= sorted(glob.glob(os.path.join(annotsPath, '*.json')), key=sortCriterion)
        self.output_folder_path=output_folder_path
    
    def crop_images(self):
        """
        Crop images using annotations.

        Loads images from `folder_path`, reads the corresponding annotations in json format
        from `annotsPath`, crops the images using the bounding boxes in the annotations, and
        saves the cropped images in the `output_folder_path`.
        """
        i = 0
        for image_file in self.image_files:
            # Load the image
            img = cv2.imread(os.path.join(image_file))
            with open(self.annotsPath[i], mode='r') as f:
                jsonObject = json.loads(f.read())
                resultsArr = []
                for each in jsonObject.get('objects'):
                    resultsArr.append({
                        'bbox': each.get('points').get('exterior'),
                    })

                for j, box in enumerate(resultsArr):
                    x1, y1= int(box['bbox'][0][0]), int(box['bbox'][0][1])
                    x2,y2= int(box['bbox'][1][0]), int(box['bbox'][1][1])
                    cropped_image = img[y1:y2, x1:x2]
                    # Save the cropped image
                    image_name=image_file.split('\\', 1)[1]
                    cv2.imwrite(os.path.join(self.output_folder_path, f"{j+1}_{image_name}"), cropped_image)
                    

            i += 1

# Define the folder path
imagesPath = "fsoco_sample/images"
annotsPath = "fsoco_sample/bounding_boxes"
outputPath="test"
# Define the coordinates for multiple boxes
# Create an instance of the dataset loader
dataset_loader = DatasetLoader(imagesPath,annotsPath,outputPath)
# Crop the images
dataset_loader.crop_images()
